# Whats inside the downloaded data=>

## Inspecting the nc files=>

In [1]:
import xarray as xr

# Path to one downloaded file
file_path = "/home/nilay/SSD Data Downloads/ERA5 Hourly 2025 October/ERA5_PL_2025_10_01.nc"

ds = xr.open_dataset(file_path)

print(ds)              # Dataset summary
print("\nVariables:")
print(list(ds.data_vars))

print("\nDimensions:")
print(ds.dims)

print("\nCoordinates:")
print(list(ds.coords))


<xarray.Dataset> Size: 4GB
Dimensions:         (valid_time: 24, pressure_level: 8, latitude: 721,
                     longitude: 1440)
Coordinates:
  * valid_time      (valid_time) datetime64[ns] 192B 2025-10-01 ... 2025-10-0...
  * pressure_level  (pressure_level) float64 64B 1e+03 925.0 ... 400.0 300.0
  * latitude        (latitude) float64 6kB 90.0 89.75 89.5 ... -89.75 -90.0
  * longitude       (longitude) float64 12kB 0.0 0.25 0.5 ... 359.2 359.5 359.8
    number          int64 8B ...
    expver          (valid_time) <U4 384B ...
Data variables:
    u               (valid_time, pressure_level, latitude, longitude) float32 797MB ...
    v               (valid_time, pressure_level, latitude, longitude) float32 797MB ...
    w               (valid_time, pressure_level, latitude, longitude) float32 797MB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 797MB ...
    z               (valid_time, pressure_level, latitude, longitude) float32 797MB ...
At

In [6]:
pip install netCDF4


Note: you may need to restart the kernel to use updated packages.


In [14]:
import zipfile
import os

file_path = "/home/nilay/SSD Data Downloads/ERA5 Hourly 2025 October/ERA5_SL_2025_10_01.nc"
folder = os.path.dirname(file_path)

with zipfile.ZipFile(file_path, 'r') as z:
    z.extractall(folder)

os.remove(file_path)

print("✅ Extracted ERA5 file")


✅ Extracted ERA5 file


# Unzipping the total precipitation files =>

In [18]:
import os
import glob
import zipfile
import shutil

# ================================
# USER PATHS
# ================================

input_dir  = "/home/nilay/SSD Data Downloads/ERA5 Hourly 2025 October"
output_dir = "/home/nilay/SSD Data Downloads/ERA5 October Hourly Unziped"

os.makedirs(output_dir, exist_ok=True)

# ================================
# PROCESS FILES
# ================================

files = sorted(glob.glob(os.path.join(input_dir, "*.nc")))

print(f"Found {len(files)} files\n")

for f in files:

    base_name = os.path.basename(f)
    print(f"Processing → {base_name}")

    # Check if ZIP disguised as nc
    if zipfile.is_zipfile(f):

        with zipfile.ZipFile(f, 'r') as z:

            for member in z.namelist():

                extracted_path = z.extract(member, output_dir)

                # Rename to match original date
                new_name = os.path.join(output_dir, base_name)
                shutil.move(extracted_path, new_name)

                print(f"   ✅ Saved → {new_name}")

    else:
        print("   ⚠ Not a ZIP — skipping")

print("\n🎯 All files extracted and renamed!")


Found 63 files

Processing → ERA5_PL_2025_10_01.nc
   ⚠ Not a ZIP — skipping
Processing → ERA5_PL_2025_10_02.nc
   ⚠ Not a ZIP — skipping
Processing → ERA5_PL_2025_10_03.nc
   ⚠ Not a ZIP — skipping
Processing → ERA5_PL_2025_10_04.nc
   ⚠ Not a ZIP — skipping
Processing → ERA5_PL_2025_10_05.nc
   ⚠ Not a ZIP — skipping
Processing → ERA5_PL_2025_10_06.nc
   ⚠ Not a ZIP — skipping
Processing → ERA5_PL_2025_10_07.nc
   ⚠ Not a ZIP — skipping
Processing → ERA5_PL_2025_10_08.nc
   ⚠ Not a ZIP — skipping
Processing → ERA5_PL_2025_10_09.nc
   ⚠ Not a ZIP — skipping
Processing → ERA5_PL_2025_10_10.nc
   ⚠ Not a ZIP — skipping
Processing → ERA5_PL_2025_10_11.nc
   ⚠ Not a ZIP — skipping
Processing → ERA5_PL_2025_10_12.nc
   ⚠ Not a ZIP — skipping
Processing → ERA5_PL_2025_10_13.nc
   ⚠ Not a ZIP — skipping
Processing → ERA5_PL_2025_10_14.nc
   ⚠ Not a ZIP — skipping
Processing → ERA5_PL_2025_10_15.nc
   ⚠ Not a ZIP — skipping
Processing → ERA5_PL_2025_10_16.nc
   ⚠ Not a ZIP — skipping
Processi

# inspecting the total precipitation files =>

In [1]:
import xarray as xr
import glob

files = glob.glob("/home/nilay/SSD Data Downloads/ERA5 October Hourly Unziped/ERA5_SL_2025_10_26.nc")

ds = xr.open_mfdataset(files, engine="netcdf4")

print(ds)


<xarray.Dataset> Size: 100MB
Dimensions:     (valid_time: 24, latitude: 721, longitude: 1440)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 192B 2025-10-26 ... 2025-10-26T23...
  * latitude    (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    number      int64 8B ...
    expver      (valid_time) <U4 384B dask.array<chunksize=(24,), meta=np.ndarray>
Data variables:
    tp          (valid_time, latitude, longitude) float32 100MB dask.array<chunksize=(12, 361, 720), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2026-02-09T22:22 GRIB to CDM+CF via cfgrib-0.9.1...


# 